In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from datetime import datetime, date, timedelta
import pandas as pd
from os import listdir

In [2]:
def removeNonAscii(s): return "".join(filter(lambda x: ord(x)<128, s))


In [162]:
#grabs the data from the website

currentday = date(2016, 1, 1)

while currentday < date(2016, 7, 2):
    print(currentday)

    y = currentday.year
    m = currentday.month
    d = currentday.day

    url = 'https://www.wunderground.com/history/airport/KNYC/' + str(y) + '/' + str(m) + '/' + str(d) + '/DailyHistory.html?req_city=New+York&req_state=NY&req_statename=New+York&reqdb.zip=10001&reqdb.magic=7&reqdb.wmo=99999'
    a = urlopen(url)
    s = BeautifulSoup(a, 'html.parser')
    
    with open('weather_files/' + str(y) + '_' + str(m) + '_' + str(d) + '.html', 'w') as h:
        h.write(str(s))
        
    currentday += timedelta(1)



2016-01-01
2016-01-02
2016-01-03
2016-01-04
2016-01-05
2016-01-06
2016-01-07
2016-01-08
2016-01-09
2016-01-10
2016-01-11
2016-01-12
2016-01-13
2016-01-14
2016-01-15
2016-01-16
2016-01-17
2016-01-18
2016-01-19
2016-01-20
2016-01-21
2016-01-22
2016-01-23
2016-01-24
2016-01-25
2016-01-26
2016-01-27
2016-01-28
2016-01-29
2016-01-30
2016-01-31
2016-02-01
2016-02-02
2016-02-03
2016-02-04
2016-02-05
2016-02-06
2016-02-07
2016-02-08
2016-02-09
2016-02-10
2016-02-11
2016-02-12
2016-02-13
2016-02-14
2016-02-15
2016-02-16
2016-02-17
2016-02-18
2016-02-19
2016-02-20
2016-02-21
2016-02-22
2016-02-23
2016-02-24
2016-02-25
2016-02-26
2016-02-27
2016-02-28
2016-02-29
2016-03-01
2016-03-02
2016-03-03
2016-03-04
2016-03-05
2016-03-06
2016-03-07
2016-03-08
2016-03-09
2016-03-10
2016-03-11
2016-03-12
2016-03-13
2016-03-14
2016-03-15
2016-03-16
2016-03-17
2016-03-18
2016-03-19
2016-03-20
2016-03-21
2016-03-22
2016-03-23
2016-03-24
2016-03-25
2016-03-26
2016-03-27
2016-03-28
2016-03-29
2016-03-30
2016-03-31

In [3]:
# Goes through the files and grabs the data we want, then saves it to a csv

files = listdir('weather_files')

dates = []
temps = []
precip = []
windspeed = []
pressure = []
humidity = []
events = []

dailysnow = []
dailyprecip = []

conditions = []

for ele in files:
    [y, m, d] = [int(i) for i in ele[:-5].split('_')]

    s = BeautifulSoup(open('weather_files/' + ele), 'html.parser')
    
    historyrows = s.find(id='historyTable').find('tbody').find_all('tr')
    
    titles = [ele.find_all('td')[0].text.strip() for ele in historyrows]
    day_snow_ind = titles.index('Snow') + 1
    day_precip_ind = titles.index('Precipitation') + 1
    
    daysnow = historyrows[day_snow_ind].find_all('td')[1].text.strip()
    daysnow = removeNonAscii(daysnow)[:-2]
    
    try:
        daysnow = float(daysnow)
    except:
        daysnow = 0.0

    dayprecip = historyrows[day_precip_ind].find_all('td')[1].text.strip()
    dayprecip = removeNonAscii(dayprecip)[:-2]
    
    try:
        dayprecip = float(dayprecip)
    except:
        dayprecip = 0.0

    
    rows = s.find(id='observations_details').find('tbody').find_all('tr')
        
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        
        if len(cols) == 12:
            precipind = 9
            pressureind = 4
            humidityind = 3
            windspeedind = 7
            eventind = 10
            condind = 11
        else:
            precipind = 10
            pressureind = 5
            humidityind = 4
            windspeedind = 8
            eventind = 11
            condind = 12
        try:
            tempstr = removeNonAscii(cols[1])
            temps.append(float(tempstr[:-1]))
            precipstr = removeNonAscii(cols[precipind])
            precip.append(float(precipstr[:-2]) if precipstr[:-3] else 0)
            try:
                pressure.append(float(removeNonAscii(cols[pressureind])[:-2]))
            except:
                pressure.append('')
            humidity.append(float(removeNonAscii(cols[humidityind])[:-1]))
            try:
                windspeed.append(float(removeNonAscii(cols[windspeedind])[:-3]))
            except:
                windspeed.append('')
            
            events.append(cols[eventind])
            
            dailysnow.append(float(daysnow))
            dailyprecip.append(float(dayprecip))
            
            conditions.append(cols[condind])
            
            dates.append(datetime.strptime(str(y) + ' ' + str(m) + ' ' + str(d) + ' ' + cols[0], '%Y %m %d %I:%M %p'))
        except:
            print(len(cols))
            print(cols)
    
weatherdata = pd.DataFrame({'events': events,
                            'dailysnow': dailysnow,
                            'dailyprecip': dailyprecip,
                            'conditions': conditions,
                            'pressure': pressure,
                            'precip': precip, 
                            'humidity': humidity, 
                            'windspeed': windspeed,
                            'temp':temps, 
                            'timestamp':dates,})

weatherdata['fog'] = weatherdata['events'].apply(lambda x: 1 if 'Fog' in x else 0)
weatherdata['rain'] = weatherdata['events'].apply(lambda x: 1 if 'Rain' in x else 0)
weatherdata['snow'] = weatherdata['events'].apply(lambda x: 1 if 'Snow' in x else 0)

weatherdata = weatherdata.sort_values(by='timestamp')
del weatherdata['events']

col = ['timestamp', 'temp', 'windspeed', 'humidity', 'precip', 'pressure', 'conditions', 'dailyprecip', 'dailysnow', 'fog', 'rain', 'snow']
weatherdata[col].to_csv('weatherdata.csv', index=False)

13
['7:00 AM', '-', '-', '-', 'N/A%', '-', '-', 'North', '-', '-', 'N/A', '', 'Unknown']
12
['7:00 AM', '-', '-', 'N/A%', '-', '-', 'North', '-', '-', 'N/A', '', 'Unknown']
13
['9:00 AM', '-', '-', '-', 'N/A%', '-', '-', 'North', '-', '-', 'N/A', '', 'Unknown']
13
['6:00 AM', '-', '-', '-', 'N/A%', '-', '-', 'North', '-', '-', 'N/A', '', 'Unknown']
